In [102]:
# https://www.kaggle.com/c/ashrae-energy-prediction/data
# {0: electricity, 1: chilledwater, 2: steam, hotwater: 3} 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split

In [2]:
# import
building = pd.read_csv("data/building_metadata.csv")
train = pd.read_csv("data/train.csv")
weather = pd.read_csv("data/weather_train.csv")
full = (train.merge(building, on='building_id')).merge(weather, on=['site_id', 'timestamp'],how='left')
# data cleaning


# algorithm data
X = full[["meter", "site_id", "square_feet", "year_built", "floor_count", "air_temperature", "cloud_coverage", "dew_temperature", "precip_depth_1_hr", "sea_level_pressure", "wind_direction", "wind_speed"]]
for i in X.columns:
    X[i].fillna(X[i].mean(), inplace=True)

y = full["meter_reading"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# test data
test = pd.read_csv("data/test.csv")
weather_test = pd.read_csv("data/weather_test.csv")
full_test = (test.merge(building, on='building_id')).merge(weather_test, on=['site_id', 'timestamp'],how='left')
X_final = full_test[["meter", "site_id", "square_feet", "year_built", "floor_count", "air_temperature", "cloud_coverage", "dew_temperature", "precip_depth_1_hr", "sea_level_pressure", "wind_direction", "wind_speed"]]
for i in X_final.columns:
    X_final[i].fillna(X[i].mean(), inplace=True)

In [ ]:
# defining scoring metric
def RMSLE (y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))
rmsle_scorer = make_scorer(RMSLE, greater_is_better=False)

# creating the pipeline
pipe_tree = make_pipeline(DecisionTreeRegressor(random_state=1))
    
# grid search
param_grid = [{'decisiontreeregressor__max_depth':[1, 5, 10, 20, 50],
              'decisiontreeregressor__min_samples_leaf':[1, 5, 20, 50]}]

gs = GridSearchCV(estimator=pipe_tree, param_grid=param_grid, scoring=rmsle_scorer, cv=10)

# training the model
gs = gs.fit(X_train, y_train)

# best model
best_score = gs.best_score_
best_params_ = gs.best_params_
best_model = gs.best_estimator_

# final test
preds = best_model.predict(X_test)
accuracy = RMSLE(y_test, preds)

In [60]:
# grid search function
def grid_search_best_model(X, y, test_size, model, scoring_metric, param_grid, n_folds):
    # train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    # creating the pipeline
    pipe_tree = make_pipeline(model)

    gs = GridSearchCV(estimator=pipe_tree, param_grid=param_grid, scoring=rmsle_scorer, cv=10)

    # training the model
    gs = gs.fit(X_train, y_train)

    # best model
    best_score = gs.best_score_
    best_params = gs.best_params_
    best_model = gs.best_estimator_

    # final test
    preds = best_model.predict(X_test)
    accuracy = RMSLE(y_test, preds)
    
    return best_score, best_params, best_model, accuracy

# scoring function
def RMSLE (y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))
rmsle_scorer = make_scorer(RMSLE, greater_is_better=False)

# generating submission
def gen_submission(model, X, model_name):
    final_preds = model.predict(X)
    pd.DataFrame({
        "row_id": X.index,
        "meter_reading": final_preds
        }).to_csv(f"data/{model_name}_submission.csv", index=False)
    
# storing performance
performance = pd.read_csv("data/performance.csv")

In [35]:
# decision tree
model_name = "dt"
param_grid = [{'decisiontreeregressor__max_depth':[1, 5, 10, 20, 50],
                  'decisiontreeregressor__min_samples_leaf':[1, 5, 20, 50]}]
model = DecisionTreeRegressor()
n_folds = 10

# scoring and submission
best_score, best_params, best_model, accuracy = grid_search_best_model(X, y, 0.2, model, rmsle_scorer, param_grid, n_folds)
preds = best_model.predict(X_test)
validation_accuracy = RMSLE(y_test, preds)
print(f"{round(validation_accuracy, 4)} accuracy on test set for {model_name}")
gen_submission(best_model, X_final, model_name)
tmp_performance = pd.DataFrame({
    "model_name":model_name, "best_params":str(best_params), "validation_accuracy":validation_accuracy, "test_accuracy":2.79, "notes":"trained on 80% of data"
}, index=[0])
performance = performance.append(tmp_performance).reset_index(drop=True)
performance.to_csv("data/performance.csv", index=False)

In [ ]:
# random forest
model_name = "rf"
param_grid = [{'randomforestregressor__n_estimators':[50, 100, 200, 500],
              'randomforestregressor__max_features':['auto', 'sqrt'],
            'randomforestregressor__max_depth':[10, 20],
            'randomforestregressor__min_samples_split':[2, 5, 10]
              }]
model = RandomForestRegressor()
n_folds = 5

# scoring and submission
best_score, best_params, best_model, accuracy = grid_search_best_model(X, y, 0.2, model, rmsle_scorer, param_grid, n_folds)
preds = best_model.predict(X_test)
validation_accuracy = RMSLE(y_test, preds)
print(f"{round(validation_accuracy, 4)} accuracy on test set for {model_name}")
gen_submission(best_model, X_final, model_name)
tmp_performance = pd.DataFrame({
    "model_name":model_name, "best_params":str(best_params), "validation_accuracy":validation_accuracy, "test_accuracy":0, "notes":"trained on 80% of data"
}, index=[0])
performance = performance.append(tmp_performance).reset_index(drop=True)
performance.to_csv("data/performance.csv", index=False)